In [ ]:
# 따릉이 대여소 현황 데이터 조회 (OPEN API)
# 조회된 데이터를 데이터베이스에 저장 (테이블 만들기 + 데이터 저장)

# apikey=5668556e4e6f686337344d77484770

In [ ]:
# 라이브러리 준비
import requests
import json
import pandas as pd

In [4]:
api_key="5668556e4e6f686337344d77484770"
url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbCycleStationInfo/1/5/"

resp = requests.get(url)
print(resp.content.decode('utf-8'))

{"stationInfo":{"list_total_count":"3143","RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다."},"row":[{"STA_LOC":"마포구","RENT_ID":"ST-10","RENT_NO":"00108","RENT_NM":"서교동 사거리","RENT_ID_NM":"108. 서교동 사거리","HOLD_NUM":"12","STA_ADD1":"서울특별시 마포구 양화로 93","STA_ADD2":"427","STA_LAT":"37.55274582","STA_LONG":"126.91861725","START_INDEX":0,"END_INDEX":0,"RNUM":"1"},{"STA_LOC":"광진구","RENT_ID":"ST-100","RENT_NO":"00503","RENT_NM":"더샵스타시티 C동 앞","RENT_ID_NM":"503. 더샵스타시티 C동 앞","HOLD_NUM":"15","STA_ADD1":"서울특별시 광진구 아차산로 262","STA_ADD2":"더샵스타시티 C동 앞","STA_LAT":"37.53666687","STA_LONG":"127.07359314","START_INDEX":0,"END_INDEX":0,"RNUM":"2"},{"STA_LOC":"양천구","RENT_ID":"ST-1000","RENT_NO":"00729","RENT_NM":"서부식자재마트 건너편","RENT_ID_NM":"729. 서부식자재마트 건너편","HOLD_NUM":"10","STA_ADD1":"서울특별시 양천구 신정동 236 ","STA_ADD2":"서부식자재마트 건너편","STA_LAT":"37.51037979","STA_LONG":"126.86679840","START_INDEX":0,"END_INDEX":0,"RNUM":"3"},{"STA_LOC":"양천구","RENT_ID":"ST-1002","RENT_NO":"00731","RENT_NM":"서울시 도로환경관리센터","RENT_ID_NM

In [5]:
# result_dict = json.loads(resp.content.decode('utf-8'))
result_dict = resp.json()
result_dict

{'stationInfo': {'list_total_count': '3143',
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'STA_LOC': '마포구',
    'RENT_ID': 'ST-10',
    'RENT_NO': '00108',
    'RENT_NM': '서교동 사거리',
    'RENT_ID_NM': '108. 서교동 사거리',
    'HOLD_NUM': '12',
    'STA_ADD1': '서울특별시 마포구 양화로 93',
    'STA_ADD2': '427',
    'STA_LAT': '37.55274582',
    'STA_LONG': '126.91861725',
    'START_INDEX': 0,
    'END_INDEX': 0,
    'RNUM': '1'},
   {'STA_LOC': '광진구',
    'RENT_ID': 'ST-100',
    'RENT_NO': '00503',
    'RENT_NM': '더샵스타시티 C동 앞',
    'RENT_ID_NM': '503. 더샵스타시티 C동 앞',
    'HOLD_NUM': '15',
    'STA_ADD1': '서울특별시 광진구 아차산로 262',
    'STA_ADD2': '더샵스타시티 C동 앞',
    'STA_LAT': '37.53666687',
    'STA_LONG': '127.07359314',
    'START_INDEX': 0,
    'END_INDEX': 0,
    'RNUM': '2'},
   {'STA_LOC': '양천구',
    'RENT_ID': 'ST-1000',
    'RENT_NO': '00729',
    'RENT_NM': '서부식자재마트 건너편',
    'RENT_ID_NM': '729. 서부식자재마트 건너편',
    'HOLD_NUM': '10',
    'STA_ADD1': '서울특별시 양천구 신정동 236 ',
   

In [6]:
result_dict['stationInfo']['row']

[{'STA_LOC': '마포구',
  'RENT_ID': 'ST-10',
  'RENT_NO': '00108',
  'RENT_NM': '서교동 사거리',
  'RENT_ID_NM': '108. 서교동 사거리',
  'HOLD_NUM': '12',
  'STA_ADD1': '서울특별시 마포구 양화로 93',
  'STA_ADD2': '427',
  'STA_LAT': '37.55274582',
  'STA_LONG': '126.91861725',
  'START_INDEX': 0,
  'END_INDEX': 0,
  'RNUM': '1'},
 {'STA_LOC': '광진구',
  'RENT_ID': 'ST-100',
  'RENT_NO': '00503',
  'RENT_NM': '더샵스타시티 C동 앞',
  'RENT_ID_NM': '503. 더샵스타시티 C동 앞',
  'HOLD_NUM': '15',
  'STA_ADD1': '서울특별시 광진구 아차산로 262',
  'STA_ADD2': '더샵스타시티 C동 앞',
  'STA_LAT': '37.53666687',
  'STA_LONG': '127.07359314',
  'START_INDEX': 0,
  'END_INDEX': 0,
  'RNUM': '2'},
 {'STA_LOC': '양천구',
  'RENT_ID': 'ST-1000',
  'RENT_NO': '00729',
  'RENT_NM': '서부식자재마트 건너편',
  'RENT_ID_NM': '729. 서부식자재마트 건너편',
  'HOLD_NUM': '10',
  'STA_ADD1': '서울특별시 양천구 신정동 236 ',
  'STA_ADD2': '서부식자재마트 건너편',
  'STA_LAT': '37.51037979',
  'STA_LONG': '126.86679840',
  'START_INDEX': 0,
  'END_INDEX': 0,
  'RNUM': '3'},
 {'STA_LOC': '양천구',
  'RENT_ID': 'ST-100

In [7]:
df_stations = pd.DataFrame(result_dict['stationInfo']['row'])
df_stations

,STA_LOC,RENT_ID,RENT_NO,RENT_NM,RENT_ID_NM,HOLD_NUM,STA_ADD1,STA_ADD2,STA_LAT,STA_LONG,START_INDEX,END_INDEX,RNUM
0,마포구,ST-10,00108,서교동 사거리,108. 서교동 사거리,12,서울특별시 마포구 양화로 93,427,37.55274582,126.91861725,0,0,1
1,광진구,ST-100,00503,더샵스타시티 C동 앞,503. 더샵스타시티 C동 앞,15,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.53666687,127.07359314,0,0,2
2,양천구,ST-1000,00729,서부식자재마트 건너편,729. 서부식자재마트 건너편,10,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.51037979,126.86679840,0,0,3
3,양천구,ST-1002,00731,서울시 도로환경관리센터,731. 서울시 도로환경관리센터,10,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.52989960,126.87654114,0,0,4
4,양천구,ST-1003,00732,신월중학교,732. 신월중학교,10,서울특별시 양천구 화곡로 59,신월동 이마트,37.53955078,126.82830048,0,0,5
